# Appending of different words to create sentences


## Purpose of this notebook is to illustrate why we need neural methods to synthesize notebook

In [1]:
import os
os.chdir("fast_speech_synthesizer/")

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from matplotlib import pyplot as plt
import numpy as np

from tacotron.synthesizer import Synthesizer
from hparams import hparams
from tacotron.utils.text import text_to_sequence
from datasets import audio

import IPython.display as ipd
import tensorflow as tf

In [4]:
synth = Synthesizer()
checkpoint_path="logs/model.ckpt-189500"
checkpoint_path
synth.load(checkpoint_path)

Constructing model: Tacotron
Initialized Tacotron model. Dimensions (? = dynamic shape): 
  embedding:                (1, ?, 512)
  enc conv out:             (1, ?, 512)
  encoder out:              (1, ?, 512)
  decoder out:              (?, ?, 80)
  residual out:             (?, ?, 512)
  projected residual out:   (?, ?, 80)
  mel out:                  (?, ?, 80)
  <stop_token> out:         (?, ?)
Loading checkpoint: logs/model.ckpt-189500
INFO:tensorflow:Restoring parameters from logs/model.ckpt-189500


In [5]:
sample_rate=22050
cleaner_names = [x.strip() for x in hparams.cleaners.split(',')]
def synthesis(text):
    seq = text_to_sequence(text, cleaner_names)
    feed_dict = {
        synth.model.inputs: [np.asarray(seq, dtype=np.int32)],
        synth.model.input_lengths: np.asarray([len(seq)], dtype=np.int32),
    }
    mels= synth.session.run(synth.mel_outputs, feed_dict=feed_dict)
    mels = mels.reshape(-1, hparams.num_mels)
    wav = audio.inv_mel_spectrogram(mels.T)
    return wav

In [6]:
def synthesize_text(text):
    lines=text.split(".")
    output=np.asarray([])
    for each_line in lines:
        words=each_line.split(" ")
        max_words=1
        if len(words) > max_words :
            last_set=0
            while (last_set<len(words)):
                each_set=" ".join(words[last_set:last_set+max_words])+"."
                last_set+=max_words
                output=np.append(output,np.asarray(synthesis(each_set)))
        else:
            output=np.append(output,np.asarray(synthesis(each_line+".")))
    ipd.display(ipd.Audio(output,rate=22050))
    return output

In [7]:
text="Hey guys, How's the demo??"


synthesize_text(text);
ipd.display(ipd.Audio(synthesis(text),rate=22050))